In [1]:
from pprint import pprint
import pandas as pd
import numpy as np
import seaborn as sns

import praw
from datetime import datetime, timedelta
from prawcore.exceptions import Redirect

user_agent = "Scraper 1.0 by AmphibianFinal9736"
reddit = praw.Reddit(client_id="HUmX-XMSMg7y2lnN-O21DA", client_secret="e2w9HmBAsBOPwZvuHDDykxsm12PKIA", user_agent=user_agent, password="Wangyibo@11")

import nltk
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from prawcore.exceptions import Redirect

from collections import Counter
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from wordcloud import WordCloud
import matplotlib.pyplot as plt
import plotly.express as px




[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/varalam/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /Users/varalam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/varalam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def fetch_posts(subreddit_names, keywords, limit_per_subreddit=None):
    sia = SIA()

    # Lists to store sentiment scores
    posts_results = []

    for subreddit_name in subreddit_names:
        subreddit = reddit.subreddit(subreddit_name)
        print(f"\nFetching posts from r/{subreddit_name}...\n{'=' * 40}")

        try:
            for submission in subreddit.search(query=' '.join(keywords), sort='relevance', time_filter='all', limit=limit_per_subreddit):
                # Perform sentiment analysis on the title
                pol_score_post = sia.polarity_scores(submission.title)
                pol_score_post['headline'] = submission.title

                # Check if the submission has a self-text (body) and perform sentiment analysis
                if submission.is_self and submission.selftext:
                    pol_score_selftext = sia.polarity_scores(submission.selftext)
                    pol_score_post['selftext'] = submission.selftext
                    pol_score_post['selftext_sentiment'] = pol_score_selftext

                posts_results.append(pol_score_post)

        except Redirect as e:
            print(f"Redirect exception: {e}")
            continue

    return posts_results



In [3]:
def process_headlines(df, additional_words_to_remove=None):
    """
    Process headlines in a DataFrame by tokenizing, removing stopwords, punctuation, and specified words,
    and then count the occurrences of each word. Also adds sentiment labels to the DataFrame.

    Parameters:
    - df: DataFrame containing a 'headline' column.
    - additional_words_to_remove: List of additional words to remove.

    Returns:
    - DataFrame with columns 'Word', 'Frequency', and 'Label'.
    """
    # Assign sentiment labels based on the 'compound' column
    df['label'] = 0
    df.loc[df['compound'] > 0.2, 'label'] = 1
    df.loc[df['compound'] < -0.2, 'label'] = -1
    
    # Fill NaN values in the 'compound' column with 0
    df['compound'] = df['compound'].fillna(0)

    # Tokenize the comments into words
    df['tokens'] = df['headline'].apply(word_tokenize)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    df['filtered_tokens'] = df['tokens'].apply(lambda tokens: [word for word in tokens if word.lower() not in stop_words])

    # Remove punctuation
    df['filtered_tokens'] = df['filtered_tokens'].apply(lambda tokens: [word for word in tokens if word.isalnum()])

    # Additional words to remove
    words_to_remove = ['v12', 'sto', 'Lamborghini', 'Lambo', 'car', 'lambo', 'lamborghini', 'huracan', 'aventador', 'countach',
                        'diablo', 'urus', 'cars', 'ferrari', 'revuelto', 'gallardo', 'miura', 'svj']
    
    
    # Flatten the list of filtered tokens
    all_words = [word for tokens in df['filtered_tokens'] for word in tokens]

    # Count the occurrences of each word
    word_counts = Counter(all_words)

    # Get the most common words and their frequencies
    processed_headlines = pd.DataFrame(word_counts.most_common(), columns=['Word', 'Frequency'])

    # Add sentiment labels to the processed DataFrame
    processed_headlines['Label'] = df['label']


    if additional_words_to_remove:
        words_to_remove.extend(additional_words_to_remove)

    df['filtered_tokens'] = df['filtered_tokens'].apply(lambda tokens: [word.lower() for word in tokens if word.lower() not in words_to_remove])

    
    return processed_headlines

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import os

def generate_topics(df, subreddit_name, num_topics=3):
    # List of Lamborghini car models to be removed
    lamborghini_models = [
        "Huracan", "Aventador", "Urus", "Gallardo", "Murcielago", "Diablo",
        "Countach", "Jalpa", "Espada", "Islero", "Jarama", "Silhouette",
        "Urraco", "Miura", "Reventon", "Centenario", "Sian", 'personam', 'Revuelto', 'svj'
        # Add more Lamborghini models as need
    ]

    # Convert Lamborghini car models to lowercase
    lamborghini_models_lower = [model.lower() for model in lamborghini_models]

    # Tokenization and preprocessing
    stop_words = set(stopwords.words('english'))
    headline_doc = df['headline']
    headline_doc = [word_tokenize(doc.lower()) for doc in headline_doc]
    headline_doc = [
        [word for word in doc if word.isalnum() and word not in stop_words and word not in lamborghini_models_lower]
        for doc in headline_doc
    ]

    # Create TF-IDF matrix
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([' '.join(doc) for doc in headline_doc])

    # Save TF-IDF matrix
    save_path = f'{subreddit_name}_tfidf_matrix.npy'
    np.save(save_path, tfidf_matrix)

    # Apply Truncated SVD (LSA)
    lsa_model = TruncatedSVD(n_components=num_topics)
    lsa_topic_matrix = lsa_model.fit_transform(tfidf_matrix)

    # Print the top words for each topic
    terms = vectorizer.get_feature_names_out()
    topics = {}
    for i in range(num_topics):
        top_words_idx = lsa_model.components_[i].argsort()[-5:][::-1]
        top_words = [terms[idx] for idx in top_words_idx]
        topics[f'Topic {i + 1}'] = top_words

    # Return the identifier along with topics and tfidf_matrix
    return subreddit_name, (topics, tfidf_matrix)

In [12]:
import gensim
from gensim import corpora
from gensim.models import LdaModel


def train_lda_model(documents, num_topics=10, num_words=5, passes=10):
    # Tokenization and preprocessing
    stop_words = set(stopwords.words('english'))
    documents = [word_tokenize(doc.lower()) for doc in documents]
    documents = [[word for word in doc if word.isalnum() and word not in stop_words] for doc in documents]

    # Create a dictionary representation of the documents
    dictionary = corpora.Dictionary(documents)

    # Create a corpus from the documents
    corpus = [dictionary.doc2bow(doc) for doc in documents]

    # Train the LDA model
    lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=passes)

    # Print the topics
    topics = lda_model.print_topics(num_words=num_words)
    formatted_topics = []

    for topic_id, topic in topics:
        terms = [term.split('"')[1] for term in topic.split('+')]
        formatted_topic = f"Topic {topic_id}: {' + '.join(terms)}"
        formatted_topics.append(formatted_topic)

    return formatted_topics



In [22]:
# Set the subreddit and keyword
subreddit_names = ['lamborghini']
keywords = ['games', 'game', 'GAME', 'videogames', 'racinggame', 'video games', 'raced', 'race', 'league', 'tournament', 'collab', 'collaboration', 'asia', 'singapore', 'Singapore']

# Call the function and get the results
posts_results = fetch_posts(subreddit_names, keywords, limit_per_subreddit=None)

# Create a data frame
posts_df = pd.DataFrame.from_records(posts_results)

# Print the data frame
print("Posts DataFrame:")
print(posts_df)

# Example usage:
# Assuming you have a DataFrame named 'headlines_df'
processed_headlines_df = process_headlines(posts_df, additional_words_to_remove=['example', 'additional'])

# Display the processed DataFrame
print(processed_headlines_df.head(50))

# Truncated SVD (LSA) model:
# Assuming generate_topics returns a tuple with a string (subreddit name) and a dictionary of topics
result = generate_topics(posts_df, subreddit_name='lamborghini')
topics, tfidf_matrix = result[1]  # Access the second element of the tuple

# Iterate over topics
for topic, top_words in topics.items():
    print(f'{topic}: {top_words}')


# LDA model:
headline_topics = train_lda_model(posts_df['headline'])
for topic in headline_topics:
    print(topic)




Fetching posts from r/lamborghini...
Posts DataFrame:
   neg    neu    pos  compound  \
0  0.0  0.734  0.266    0.4404   
1  0.0  0.769  0.231    0.3400   
2  0.0  1.000  0.000    0.0000   
3  0.0  1.000  0.000    0.0000   
4  0.0  1.000  0.000    0.0000   
5  0.0  1.000  0.000    0.0000   
6  0.0  1.000  0.000    0.0000   

                                            headline  \
0  Retro Racing Game - World Record Seems Easy to...   
1  Any of you play (sim) racing games with Lambor...   
2  Asphalt 8: Airborne Part 162 | Lamborghini Ven...   
3                                       Need to khow   
4                     Exploring Lambos in Videogames   
5  Lamborghini Jarama 1970 - Ferruccio Lamborghin...   
6    Made myself a Lamborghini chair for sim racing.   

                                            selftext  \
0  My buddy made a simple retro racing game and s...   
1  I myself play Assetto Corsa Competizione where...   
2                                                NaN   

In [23]:
# Set the subreddit and keyword
subreddit_names = ['askSingapore']
keywords = ['games', 'game', 'GAME', 'videogames', 'racinggame', 'video games', 'raced', 'race', 'league', 'tournament', 'collab', 'collaboration', 'asia', 'singapore', 'Singapore']

# Call the function and get the results
posts_results = fetch_posts(subreddit_names, keywords, limit_per_subreddit=None)

# Create a data frame
posts_df = pd.DataFrame.from_records(posts_results)

# Print the data frame
print("Posts DataFrame:")
print(posts_df)

# Example usage:
# Assuming you have a DataFrame named 'headlines_df'
processed_headlines_df = process_headlines(posts_df, additional_words_to_remove=['example', 'additional'])

# Display the processed DataFrame
print(processed_headlines_df.head(50))

# Truncated SVD (LSA) model:
# Assuming generate_topics returns a tuple with a string (subreddit name) and a dictionary of topics
result = generate_topics(posts_df, subreddit_name='lamborghini')
topics, tfidf_matrix = result[1]  # Access the second element of the tuple

# Iterate over topics
for topic, top_words in topics.items():
    print(f'{topic}: {top_words}')


# LDA model:
headline_topics = train_lda_model(posts_df['headline'])
for topic in headline_topics:
    print(topic)






Fetching posts from r/askSingapore...
Posts DataFrame:
     neg    neu    pos  compound  \
0    0.0  1.000  0.000    0.0000   
1    0.0  1.000  0.000    0.0000   
2    0.0  1.000  0.000    0.0000   
3    0.0  1.000  0.000    0.0000   
4    0.0  1.000  0.000    0.0000   
..   ...    ...    ...       ...   
231  0.0  0.674  0.326    0.4404   
232  0.0  1.000  0.000    0.0000   
233  0.0  1.000  0.000    0.0000   
234  0.0  1.000  0.000    0.0000   
235  0.0  0.385  0.615    0.4939   

                                              headline  \
0                     Will Singapore be your end game?   
1                             Dating game in Singapore   
2                Video game beta testers in Singapore?   
3                Game development courses in Singapore   
4    If Singapore’s education system were a game, w...   
..                                                 ...   
231        Need advice on getting better NS vocations.   
232                     What should I do with m

In [24]:
# Set the subreddit and keyword
subreddit_names = ['simracing']
keywords = ['lamborghini', 'Lamborghini', 'Lambo', 'lambo', 'LAMBORGHINI', 'singapore', 'Singapore']
# Call the function and get the results
posts_results = fetch_posts(subreddit_names, keywords, limit_per_subreddit=None)

# Create a data frame
posts_df = pd.DataFrame.from_records(posts_results)

# Print the data frame
print("Posts DataFrame:")
print(posts_df)

# Example usage:
# Assuming you have a DataFrame named 'headlines_df'
processed_headlines_df = process_headlines(posts_df, additional_words_to_remove=['example', 'additional'])

# Display the processed DataFrame
print(processed_headlines_df.head(50))

# Truncated SVD (LSA) model:
# Assuming generate_topics returns a tuple with a string (subreddit name) and a dictionary of topics
result = generate_topics(posts_df, subreddit_name='simracing')
topics, tfidf_matrix = result[1]  # Access the second element of the tuple

# Iterate over topics
for topic, top_words in topics.items():
    print(f'{topic}: {top_words}')



# LDA model:
headline_topics = train_lda_model(posts_df['headline'])
for topic in headline_topics:
    print(topic)






Fetching posts from r/simracing...
Posts DataFrame:
       neg    neu    pos  compound  \
0    0.000  1.000  0.000    0.0000   
1    0.000  1.000  0.000    0.0000   
2    0.000  1.000  0.000    0.0000   
3    0.134  0.642  0.225    0.4019   
4    0.000  0.804  0.196    0.4449   
..     ...    ...    ...       ...   
174  0.000  0.462  0.538    0.7906   
175  0.000  1.000  0.000    0.0000   
176  0.000  1.000  0.000    0.0000   
177  0.000  1.000  0.000    0.0000   
178  0.000  0.879  0.121    0.0258   

                                              headline  \
0    How my perspective on simracing changed after ...   
1                            NEW Lamborghini SC24 LMDH   
2    My First Ever Racing Game - Automobili Lamborg...   
3    Best sounding car in the game… Lamborghini R-G...   
4    Cant have shit in Detroit .They took my Lambor...   
..                                                 ...   
174    The Best Sims For Various Popular Racing Series   
175          Assetto Corsa

In [30]:
# Set the subreddit and keyword
subreddit_names = ['rocketracing']
keywords = ['lamborghini', 'Lamborghini', 'Lambo', 'lambo', 'LAMBORGHINI', 'singapore', 'Singapore']
# Call the function and get the results
posts_results = fetch_posts(subreddit_names, keywords, limit_per_subreddit=None)

# Create a data frame
posts_df = pd.DataFrame.from_records(posts_results)

# Print the data frame
print("Posts DataFrame:")
print(posts_df)

# Example usage:
# Assuming you have a DataFrame named 'headlines_df'
processed_headlines_df = process_headlines(posts_df, additional_words_to_remove=['lamborghini', 'Lamborghini', 'Lambo', 'lambo', 'LAMBORGHINI', 'livery'])

# Display the processed DataFrame
print(processed_headlines_df.head(50))

# Truncated SVD (LSA) model:
# Assuming generate_topics returns a tuple with a string (subreddit name) and a dictionary of topics
result = generate_topics(posts_df, subreddit_name='simracing')
topics, tfidf_matrix = result[1]  # Access the second element of the tuple

# Iterate over topics
for topic, top_words in topics.items():
    print(f'{topic}: {top_words}')



# LDA model:
headline_topics = train_lda_model(posts_df['headline'])
for topic in headline_topics:
    print(topic)







Fetching posts from r/rocketracing...
Posts DataFrame:
      neg    neu    pos  compound  \
0   0.000  1.000  0.000    0.0000   
1   0.000  0.826  0.174    0.2263   
2   0.000  1.000  0.000    0.0000   
3   0.000  1.000  0.000    0.0000   
4   0.000  0.612  0.388    0.5859   
5   0.086  0.914  0.000   -0.3089   
6   0.000  1.000  0.000    0.0000   
7   0.182  0.818  0.000   -0.3971   
8   0.000  1.000  0.000    0.0000   
9   0.000  1.000  0.000    0.0000   
10  0.000  1.000  0.000    0.0000   
11  0.234  0.660  0.107   -0.5423   
12  0.000  1.000  0.000    0.0000   
13  0.368  0.632  0.000   -0.5413   
14  0.000  1.000  0.000    0.0000   
15  0.000  0.702  0.298    0.5256   
16  0.000  0.556  0.444    0.3400   
17  0.000  1.000  0.000    0.0000   
18  0.000  0.797  0.203    0.4215   
19  0.000  1.000  0.000    0.0000   
20  0.000  1.000  0.000    0.0000   
21  0.000  1.000  0.000    0.0000   
22  0.000  0.826  0.174    0.2732   
23  0.000  1.000  0.000    0.0000   
24  0.000  1.000  0

In [28]:
# Set the subreddit and keyword
subreddit_names = ['forza']
keywords = ['lamborghini', 'Lamborghini', 'Lambo', 'lambo', 'LAMBORGHINI', 'singapore', 'Singapore']
# Call the function and get the results
posts_results = fetch_posts(subreddit_names, keywords, limit_per_subreddit=None)

# Create a data frame
posts_df = pd.DataFrame.from_records(posts_results)

# Print the data frame
print("Posts DataFrame:")
print(posts_df)

# Example usage:
# Assuming you have a DataFrame named 'headlines_df'
processed_headlines_df = process_headlines(posts_df, additional_words_to_remove=['lamborghini', 'Lamborghini', 'Lambo', 'lambo', 'LAMBORGHINI', 'livery'])

# Display the processed DataFrame
print(processed_headlines_df.head(50))

# Truncated SVD (LSA) model:
result = generate_topics(posts_df, subreddit_name='forza')
topics, tfidf_matrix = result[1]  # Access the second element of the tuple

# Iterate over topics
for topic, top_words in topics.items():
    print(f'{topic}: {top_words}')



# LDA model:
headline_topics = train_lda_model(posts_df['headline'])
for topic in headline_topics:
    print(topic)





Fetching posts from r/forza...
Posts DataFrame:
     neg  neu  pos  compound  \
0    0.0  1.0  0.0       0.0   
1    0.0  1.0  0.0       0.0   
2    0.0  1.0  0.0       0.0   
3    0.0  1.0  0.0       0.0   
4    0.0  1.0  0.0       0.0   
..   ...  ...  ...       ...   
230  0.0  1.0  0.0       0.0   
231  0.0  1.0  0.0       0.0   
232  0.0  1.0  0.0       0.0   
233  0.0  1.0  0.0       0.0   
234  0.0  1.0  0.0       0.0   

                                              headline selftext  \
0    Do we think the new Lamborghini Countach will ...      NaN   
1                        Lamborghini Urus acceleration      NaN   
2    The Evolution of the Forza Horizon Garages wit...      NaN   
3    [Wishlist] Extraterrestrial Lamborghini Counta...      NaN   
4                     Opinion on the Lamborghini Sian?      NaN   
..                                                 ...      ...   
230  When was the 2016 Lamborghini Aventador LP 750...      NaN   
231         The legendary lime